In [ ]:
! pip install python-pptx
! pip install requests
! pip install language-tool-python


In [177]:
import requests
from pptx import Presentation
from collections import Counter
import language_tool_python

In [63]:
ppt_url = 'https://docs.google.com/presentation/d/1BVRPYon5oT-a3WGGzN6gPHfZ_k9E9UwwbqazZeY1Srg/edit?usp=sharing'.split(
    '/')
ppt_url[-1] = 'export/pptx'

ppt_url = '/'.join(ppt_url)


response = requests.get(ppt_url)
open("test.pptx", "wb").write(response.content)


2442383

In [109]:
txt_lst = []
word_count = []

shape_count = 0
slide_count = 0

font_lst = []

with open('test.pptx', 'rb') as pptx:
    prs = Presentation(pptx)
    slide_count = len(prs.slides)
    for slide in prs.slides:
        slide_txt_lst = []
        slide_word_count = []
        for shape in slide.shapes:
            if shape.has_text_frame:
                for para in shape.text_frame.paragraphs:
                    for run in para.runs:
                        font_lst.append(run.font.name)
                        slide_word_count.append(len(run.text.split(' ')))
                        slide_txt_lst.append(run.text)
            if(shape.shape_type == 13 or shape.shape_type == 6):
                shape_count += 1
        txt_lst.append(slide_txt_lst)
        word_count.append(slide_word_count)

In [ ]:
print(slide_count, shape_count)

print(Counter(font_lst))

for txt, count in zip(txt_lst, word_count):
    print(txt)
    print(count)

In [180]:
mistakes = []
mistake_exp = []
top3_suggestions = []

disable_rules = ['ARROWS', 'WHITESPACE_RULE', 'EN_COMPOUNDS']

lang_tool = language_tool_python.LanguageTool('en-US')

for idx, slide_txt in enumerate(txt_lst):
    slide_mistakes = []
    slide_mistake_exp = []
    slide_top3_suggestions = []

    for txt in slide_txt:
        if len(txt) > 2:
            predict = lang_tool.check(txt)
            for rule in predict:
                if(rule.ruleId in disable_rules):
                    continue
                if len(rule.replacements) > 0:
                    if rule.context[rule.offset : rule.offset + 1].isalnum():
                        slide_mistakes.append(
                            rule.context[rule.offset: rule.errorLength + rule.offset])
                        slide_mistake_exp.append(rule.message)
                        slide_top3_suggestions.append(rule.replacements[:3])

    mistakes.append(slide_mistakes)
    mistake_exp.append(slide_mistake_exp)
    top3_suggestions.append(slide_top3_suggestions)
lang_tool.close()

In [181]:
for i in range(slide_count):
    print(f"SLIDE {i+1}")

    for j in range(len(mistakes[i])):
        print( mistakes[i][j],
              mistake_exp[i][j], top3_suggestions[i][j])


SLIDE 1
Karthick Possible spelling mistake found. ['Kart hick']
SLIDE 2
SLIDE 3
SLIDE 4
SLIDE 5
iBug Possible spelling mistake found. ['bug']
SLIDE 6
RAVDESS Possible spelling mistake found. ['RAIDERS', 'RAVENS', 'RAVES']
SAAVES Possible spelling mistake found. ['SLAVES', 'SAVES', 'STAVES']
crossentropy Possible spelling mistake found. ['cross entropy']
SLIDE 7
ocket This sentence does not start with an uppercase letter. ['Ocket']
SLIDE 8
SLIDE 9
MFCC Possible spelling mistake found. ['FCC', 'MFC', 'M FCC']
Cepstral Possible spelling mistake found. ['Central']
IDFT Possible spelling mistake found. ['IDF', 'DFT', 'IDT']
